In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.system("cp drive/MyDrive/data.zip .")
# os.system("unzip data.zip")

In [ ]:
#import libraries
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cupy as cp
from keras.utils import np_utils
import copy

import contextlib


from scipy.sparse import csr_matrix, triu
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix
from scipy.sparse import identity
from sklearn.neighbors import NearestNeighbors

import time

!pip install Munkres
from munkres import Munkres

# by using below command, gpu is available
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

dsetname = "unknown"

In [ ]:
# Helpers
def get_custom_feat(data_dir):

    data = torch.load(data_dir)
    
    X = torch.cat([data['trainX'], data['testX']], 0)
    Y = torch.cat([data['trainY'], data['testY']], 0)

    X = X.reshape(X.shape[0], -1)
    Y = Y.reshape(Y.shape[0],)

    # dataset = dict()
    # dataset['X']=X
    # dataset['Y']=Y

    # dataloader=DataLoader(TensorDataset(X,Y),batch_size=batch_size,shuffle=True,num_workers=4)

    meta = {'dim': X.shape[1], 'nClasses': int(max(Y).item())+1}

    return X, Y, X.shape[0], meta['dim'], meta['nClasses']


def get_custom_feat_npy(data_dir_X, data_dir_Y):

    X = torch.FloatTensor(np.load(data_dir_X))
    Y = torch.LongTensor(np.load(data_dir_Y))
    
    X = X.reshape(X.shape[0], -1)
    Y = Y.reshape(Y.shape[0],)

    # dataset = dict()
    # dataset['X']=X
    # dataset['Y']=Y

    # dataloader=DataLoader(TensorDataset(X,Y),batch_size=batch_size,shuffle=True,num_workers=4)

    meta = {'dim': X.shape[1], 'nClasses': int(max(Y).item())+1}

    return X, Y, X.shape[0], meta['dim'], meta['nClasses']

In [ ]:
# # download mnist dataset from keras, then define training dataset X and its true labels Y
# from keras.datasets import mnist

# (X_tr, Y_tr), (X_tst, Y_tst) = mnist.load_data()

# (num_tr, depth, _) = X_tr.shape
# (num_tst, _, _) = X_tst.shape

# dim = depth**2

# X_tr = X_tr.reshape(num_tr, dim) # from 28*28 image to naive 784 dim feature vector 
# X_tst = X_tst.reshape(num_tst, dim)
# X = np.r_[X_tr, X_tst]
# print("Size of training dataset is", X.shape)
# X = X/255
# #X = (2*(X/255)) - 1

# Y = np.r_[Y_tr, Y_tst]
# Y = Y.reshape(1, Y.shape[0])

# C = 10 # the number of classes for MNIST

# # print(X.shape)
# # print(Y.shape)

# dsetname = "mnist"

In [ ]:
# # Make Fshion Mnist dataset
# from torchvision.datasets import FashionMNIST

# def get_fashion(data_dir = './data/fashion/', batch_size=128, shuffle = True):

#     train = FashionMNIST(root = data_dir, train = True, download = True)
#     test = FashionMNIST(root = data_dir, train = False, download = True)

#     X=torch.cat([train.data.float().view(-1,784)/255.,test.data.float().view(-1,784)/255.],0)
#     Y=torch.cat([train.targets,test.targets],0)

#     meta = {'dim': 784, 'nClasses': 10}

#     return X, Y, X.shape[0], meta['dim'], meta['nClasses']

# X, Y, _, dim, C = get_fashion()

# X = X.detach().cpu().numpy()
# Y = Y.detach().cpu().numpy()
# Y = Y.reshape(1,Y.shape[0])

# print(dim)
# print(C)
# print(X.shape[0])

# dsetname = "f-mnist"

In [ ]:
# Make 20news dataset

from sklearn.datasets import fetch_20newsgroups, make_circles, make_moons, make_blobs, fetch_openml
from sklearn.feature_extraction.text import TfidfVectorizer


def get_20news():

    data = fetch_20newsgroups(subset = 'all')
    vectorizer = TfidfVectorizer(max_features = 2000, stop_words = 'english')

    X = vectorizer.fit_transform(data.data).todense().astype(np.float32)
    Y = data.target

    n_samples = X.shape[0]
    dim = X.shape[1]

    Y = Y.reshape(1, n_samples)

    n_classes = 20

    return X, Y, n_samples, dim, n_classes

X, Y, _, dim, C = get_20news()

print(dim)
print(C)
print(X.shape[0])

dsetname = "20news"

2000
20
18846


In [ ]:
# # Make Reuters10K dataset
# def get_reuters10K():
#     return get_custom_feat_npy('./IMSAT_datasets/all_dataset/reuters/10k_feature.npy', './IMSAT_datasets/all_dataset/reuters/10k_target.npy')

# X, Y, _, dim, C = get_reuters10K()

# X = X.detach().cpu().numpy()
# Y = Y.detach().cpu().numpy()
# Y = Y.reshape(1,Y.shape[0])

# print(dim)
# print(C)
# print(X.shape[0])

# dsetname = "reuters10k"

In [ ]:
# # Make CIFAR10 dataset
# # Features extracted by Zhang following IMSAT paper
# def get_CIFAR10_zhang():
#     return get_custom_feat('./CIFAR10_feat/data.pkl')

# # Features provided with IMSAT code
# # Zhang cannot get reported performance with fixed eps; did not try for adaptive epsilon.
# def get_CIFAR10_IMSAT():

#     PATH = './IMSAT_datasets/all_dataset/cifar/'

#     y_train_ul = np.load(PATH+'train_labels.npy').astype(np.int32)
#     y_test = np.load(PATH+'test_labels.npy').astype(np.int32)
#     y_whole = np.concatenate((y_train_ul, y_test), axis = 0)
#     x_whole = np.load(PATH+'resnet.npz')['arr_0']

#     X = torch.Tensor(x_whole).reshape(x_whole.shape[0], -1)
#     Y = torch.LongTensor(y_whole).reshape(y_whole.shape[0])

#     meta = {'dim': X.shape[1], 'nClasses': 10}

#     return X, Y, X.shape[0], meta['dim'], meta['nClasses']

# X, Y, _, dim, C = get_CIFAR10_IMSAT()

# X = X.detach().cpu().numpy()
# Y = Y.detach().cpu().numpy()
# Y = Y.reshape(1,Y.shape[0])

# print(dim)
# print(C)
# print(X.shape[0])

# dsetname = "cifar10"

In [ ]:
# # Make CIFAR100 dataset
# # Features extracted by Zhang following IMSAT paper
# def get_CIFAR100_zhang():
#     return get_custom_feat('./CIFAR100_feat/data.pkl')

# # Features provided with IMSAT code
# def get_CIFAR100_IMSAT():

#     PATH = './IMSAT_datasets/all_dataset/cifar100/'

#     y_whole = np.load(PATH + 'y.npy')
#     x_whole = np.load(PATH+'resnet.npz')['arr_0']

#     X = torch.Tensor(x_whole).reshape(x_whole.shape[0], -1)
#     Y = torch.LongTensor(y_whole).reshape(y_whole.shape[0])

#     meta = {'dim': X.shape[1], 'nClasses': 100}

#     return X, Y, X.shape[0], meta['dim'], meta['nClasses']


# X, Y, _, dim, C = get_CIFAR100_IMSAT()

# X = X.detach().cpu().numpy()
# Y = Y.detach().cpu().numpy()
# Y = Y.reshape(1,Y.shape[0])

# print(dim)
# print(C)
# print(X.shape[0])

# dsetname = "cifar100"

In [ ]:
# # Make SVHN dataset
# def get_svhn():
    
#     PATH = './IMSAT_datasets/all_dataset/svhn/'

#     x_train_ul = np.load(PATH+'train_feature.npy').astype(np.float32)
#     x_test = np.load(PATH+'test_feature.npy').astype(np.float32)
#     y_train_ul = np.load(PATH+'train_target.npy').astype(np.int32)
#     y_test = np.load(PATH+'test_target.npy').astype(np.int32)

#     x_whole = np.concatenate((x_train_ul, x_test), axis = 0)
#     y_whole = np.concatenate((y_train_ul, y_test), axis = 0)

#     X = torch.Tensor(x_whole).reshape(x_whole.shape[0], -1)
#     Y = torch.LongTensor(y_whole).reshape(y_whole.shape[0])

#     meta = {'dim': X.shape[1], 'nClasses': int(max(Y).item())+1}

#     return X, Y, X.shape[0], meta['dim'], meta['nClasses']


# X, Y, _, dim, C = get_svhn()

# X = X.detach().cpu().numpy()
# Y = Y.detach().cpu().numpy()
# Y = Y.reshape(1,Y.shape[0])

# print(dim)
# print(C)
# print(X.shape[0])

# dsetname = "svhn"

In [ ]:
# # Make Omniglot dataset
# def get_omniglot():
    
#     PATH = './IMSAT_datasets/all_dataset/omniglot/'
#     scale=   1.0 / 255.0
#     shift= - 0.0

#     augmented_data = (np.load(PATH+"augmented_omniglot_downsampled5_data.npz")['arr_0']).astype(np.float32)*scale + shift
#     augmented_target = np.load(PATH+"augmented_omniglot_downsampled5_target.npz")['arr_0']

#     X = torch.Tensor(augmented_data).reshape(augmented_data.shape[0], -1)
#     Y = torch.LongTensor(augmented_target).reshape(augmented_target.shape[0])

#     meta = {'dim': X.shape[1], 'nClasses': int(max(Y).item())+1}

#     return X, Y, X.shape[0], meta['dim'], meta['nClasses']

# X, Y, _, dim, C = get_omniglot()

# X = X.detach().cpu().numpy()
# Y = Y.detach().cpu().numpy()
# Y = Y.reshape(1,Y.shape[0])

# print(dim)
# print(C)
# print(X.shape[0])

# dsetname = "omniglot"

In [ ]:
# # Make STL dataset

# def get_imsat_stl10(batch_size=128):

#     PATH = './IMSAT_datasets/all_dataset/stl/'
#     label = np.fromfile(PATH+'train_y.bin', dtype=np.uint8)
#     test_label = np.fromfile(PATH+'test_y.bin', dtype=np.uint8)
#     y_whole = (np.concatenate((label, test_label), axis = 0) - 1).astype(np.int32)
#     x_whole = np.load(PATH + 'resnet.npz')['arr_0']
#     X = torch.Tensor(x_whole).reshape(x_whole.shape[0], -1)
#     Y = torch.LongTensor(y_whole).reshape(y_whole.shape[0])
#     meta = {'dim': X.shape[1], 'nClasses': 10}

#     return X, Y, X.shape[0], meta['dim'], meta['nClasses']

# X, Y, _, dim, C = get_imsat_stl10()

# X = X.detach().cpu().numpy()
# Y = Y.detach().cpu().numpy()
# Y = Y.reshape(1,Y.shape[0])

# print(dim)
# print(C)
# print(X.shape[0])

# dsetname = "stl"

In [ ]:

def conditional_entropy(soft):
    loss = torch.sum(-soft*torch.log(soft + 1e-8)) / soft.shape[0]
    return loss


def entropy(soft):
    avg_soft = torch.mean(soft, 0, True) 
    loss = -torch.sum(avg_soft * torch.log(avg_soft + 1e-8))
    return loss


def kl(p, q):
    loss = torch.sum(p * torch.log((p + 1e-8) / (q + 1e-8))) / p.shape[0]
    return loss






@contextlib.contextmanager
def _disable_tracking_bn_stats(model):

    def switch_attr(m):
        if hasattr(m, 'track_running_stats'):
            m.track_running_stats ^= True
            
    model.apply(switch_attr)
    yield
    model.apply(switch_attr)


def _l2_normalize(d):
    d_reshaped = d.view(d.shape[0], -1, *(1 for _ in range(d.dim() - 2)))
    d /= torch.norm(d_reshaped, dim=1, keepdim=True) + 1e-8
    return d


def return_vat_Loss(model, x, xi, eps):

    optimizer.zero_grad()
    
    with _disable_tracking_bn_stats(model):
        with torch.no_grad():
            target = torch.softmax(model(x), 1) 
        
        d = torch.randn(x.shape).to(dev)
        d = _l2_normalize(d)
        d.requires_grad_()
        out_vadv = model(x + xi*d)
        hat = torch.softmax(out_vadv, 1)
        adv_distance = kl(target, hat)

        adv_distance.backward()
        
        d = _l2_normalize(d.grad)
        r_adv = eps * d
        out_vadv = model(x + r_adv)
        hat = torch.softmax(out_vadv, 1)
        R_vat = kl(target, hat)

    return R_vat



def SiameseLoss(model, soft_out1, x_agm, t1, t2, r):

    m = soft_out1.shape[0]
    a = np.ones((m,m)) - np.eye(m)
    a = csr_matrix(a)
   
    neg_idx_i, neg_idx_j = a.nonzero()
    del a

    l_neg = neg_idx_i.shape[0]
    num_neg_pairs = int(l_neg*r)

     
    with _disable_tracking_bn_stats(model):
        soft_out2 = torch.softmax(model(x_agm), 1)

        s = np.random.choice(list(range(l_neg)), size=num_neg_pairs, replace=False)

        neg_ip = soft_out1[neg_idx_i[s],:] * soft_out2[neg_idx_j[s],:]
        neg_ip = torch.sum(neg_ip, 1)
        neg_loss = torch.log(1 + t2*(1-neg_ip))
        neg_loss = -torch.sum(neg_loss) / num_neg_pairs


        pos_ip = soft_out1 * soft_out2
        pos_ip = torch.sum(pos_ip, 1)
        pos_loss = torch.log(1 + t1*(1-pos_ip))
        pos_loss = torch.sum(pos_loss) / m



    return pos_loss, neg_loss, soft_out2










def ReturnACC(cluster, target_cluster, k):
    """ Compute error between cluster and target cluster
    :param cluster: proposed cluster
    :param target_cluster: target cluster
    k: number of classes
    :return: error
    """
    n = np.shape(target_cluster)[0]
    M = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            M[i][j] = np.sum(np.logical_and(cluster == i, target_cluster == j))
    m = Munkres()
    indexes = m.compute(-M)
    corresp = []
    for i in range(k):
        corresp.append(indexes[i][1])
    pred_corresp = [corresp[int(predicted)] for predicted in cluster]
    acc = np.sum(pred_corresp == target_cluster) / float(len(target_cluster))
    return acc 



def get_agm(x, idx_itr, knn_idx):
    knn_idx_itr = knn_idx[idx_itr,:]
    for i in range(knn_idx_itr.shape[0]):
      v = np.random.permutation(knn_idx_itr[i,:])
      knn_idx_itr[i,:] = v

    v = np.random.choice(list(range(knn_idx_itr.shape[1])), size=1, replace=False)
    idx0 = knn_idx_itr[:,v[0]]
    x_agm0 = x[idx0,:]
    
    return x_agm0, idx0



In [ ]:
# ## Important two classes (annoy and nmslib) to conduct approximate knn
# ## the below classes (NMSlibTransformer and AnnoyTransformer) are copied and pasted
# ## from https://scikit-learn.org/stable/auto_examples/neighbors/approximate_nearest_neighbors.html

# !pip install annoy nmslib

# # Author: Tom Dupre la Tour
# #
# # License: BSD 3 clause
# import time
# import sys

# try:
#     import annoy
# except ImportError:
#     print("The package 'annoy' is required to run this example.")
#     sys.exit()

# try:
#     import nmslib
# except ImportError:
#     print("The package 'nmslib' is required to run this example.")
#     sys.exit()

# import matplotlib.pyplot as plt
# from matplotlib.ticker import NullFormatter
# from scipy.sparse import csr_matrix

# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.neighbors import KNeighborsTransformer
# from sklearn.utils._testing import assert_array_almost_equal
# from sklearn.datasets import fetch_openml
# from sklearn.pipeline import make_pipeline
# from sklearn.manifold import TSNE
# from sklearn.utils import shuffle

# print(__doc__)


# class NMSlibTransformer(TransformerMixin, BaseEstimator):
#     """Wrapper for using nmslib as sklearn's KNeighborsTransformer"""

#     def __init__(self, n_neighbors=5, metric='euclidean', method='sw-graph',
#                  n_jobs=1):
#         self.n_neighbors = n_neighbors
#         self.method = method
#         self.metric = metric
#         self.n_jobs = n_jobs

#     def fit(self, X):
#         self.n_samples_fit_ = X.shape[0]

#         # see more metric in the manual
#         # https://github.com/nmslib/nmslib/tree/master/manual
#         space = {
#             'sqeuclidean': 'l2',
#             'euclidean': 'l2',
#             'cosine': 'cosinesimil',
#             'l1': 'l1',
#             'l2': 'l2',
#         }[self.metric]

#         self.nmslib_ = nmslib.init(method=self.method, space=space)
#         self.nmslib_.addDataPointBatch(X)
#         self.nmslib_.createIndex()
#         return self

#     def transform(self, X):
#         n_samples_transform = X.shape[0]

#         # For compatibility reasons, as each sample is considered as its own
#         # neighbor, one extra neighbor will be computed.
#         n_neighbors = self.n_neighbors + 1

#         results = self.nmslib_.knnQueryBatch(X, k=n_neighbors,
#                                              num_threads=self.n_jobs)
#         indices, distances = zip(*results)
#         indices, distances = np.vstack(indices), np.vstack(distances)

#         if self.metric == 'sqeuclidean':
#             distances **= 2

#         indptr = np.arange(0, n_samples_transform * n_neighbors + 1,
#                            n_neighbors)
#         kneighbors_graph = csr_matrix((distances.ravel(), indices.ravel(),
#                                        indptr), shape=(n_samples_transform,
#                                                        self.n_samples_fit_))

#         return kneighbors_graph


# class AnnoyTransformer(TransformerMixin, BaseEstimator):
#     """Wrapper for using annoy.AnnoyIndex as sklearn's KNeighborsTransformer"""

#     def __init__(self, n_neighbors=5, metric='euclidean', n_trees=10,
#                  search_k=-1):
#         self.n_neighbors = n_neighbors
#         self.n_trees = n_trees
#         self.search_k = search_k
#         self.metric = metric

#     def fit(self, X):
#         self.n_samples_fit_ = X.shape[0]
#         metric = self.metric if self.metric != 'sqeuclidean' else 'euclidean'
#         self.annoy_ = annoy.AnnoyIndex(X.shape[1], metric=metric)
#         for i, x in enumerate(X):
#             self.annoy_.add_item(i, x.tolist())
#         self.annoy_.build(self.n_trees)
#         return self

#     def transform(self, X):
#         return self._transform(X)

#     def fit_transform(self, X, y=None):
#         return self.fit(X)._transform(X=None)

#     def _transform(self, X):
#         """As `transform`, but handles X is None for faster `fit_transform`."""

#         n_samples_transform = self.n_samples_fit_ if X is None else X.shape[0]

#         # For compatibility reasons, as each sample is considered as its own
#         # neighbor, one extra neighbor will be computed.
#         n_neighbors = self.n_neighbors + 1

#         indices = np.empty((n_samples_transform, n_neighbors),
#                            dtype=np.int)
#         distances = np.empty((n_samples_transform, n_neighbors))

#         if X is None:
#             for i in range(self.annoy_.get_n_items()):
#                 ind, dist = self.annoy_.get_nns_by_item(
#                     i, n_neighbors, self.search_k, include_distances=True)

#                 indices[i], distances[i] = ind, dist
#         else:
#             for i, x in enumerate(X):
#                 indices[i], distances[i] = self.annoy_.get_nns_by_vector(
#                     x.tolist(), n_neighbors, self.search_k,
#                     include_distances=True)

#         if self.metric == 'sqeuclidean':
#             distances **= 2

#         indptr = np.arange(0, n_samples_transform * n_neighbors + 1,
#                            n_neighbors)
#         kneighbors_graph = csr_matrix((distances.ravel(), indices.ravel(),
#                                        indptr), shape=(n_samples_transform,
#                                                        self.n_samples_fit_))

#         return kneighbors_graph

In [ ]:
import os

start = time.time()

alpha = 0.25
K = 200 # Number of neighbors

if X.shape[0] > 5*10**5:
  nms = NMSlibTransformer(n_neighbors = K) # Compute approximated knn graph
  Knn_graph = nms.fit_transform(X)

  # define adaptie radius for VAT
  Knn_dist = Knn_graph.data.reshape(X.shape[0],K+1)
  R = alpha*Knn_dist[:,K]
  R = R.reshape(X.shape[0],1) 

  del Knn_graph, Knn_dist

  end = time.time()
  print(f"{end-start} seconds by Approximated KNN.")

else:

  knncachestr = "%s-k%d.npy" % (dsetname, K+1)
  
  if dsetname != "unknown" and os.path.exists(knncachestr):
    
    print("Loaded cached kNN from %s" % knncachestr)
    distances = np.load(knncachestr)
    indices = np.load("i" + knncachestr)
  
  else:

    nbrs = NearestNeighbors(n_neighbors=K+1, algorithm='brute').fit(X)
    distances, indices = nbrs.kneighbors(X)
    np.save(knncachestr, distances)
    np.save("i" + knncachestr, indices)

  K_vat = 10
  R = alpha*distances[:,K_vat]
  R = R.reshape(X.shape[0],1)


  end = time.time()
  print(f"{end-start} seconds by Brute-Force KNN.")



R = torch.tensor(R.astype('f')).to(dev)


X = torch.tensor(X.astype('f')).to(dev) # this unlabeled dataset (set of feature vectors) is input of IMSAT

Loaded cached kNN from 20news-k201.npy
0.016347408294677734 seconds by Brute-Force KNN.


In [ ]:
# define archtechture of MLP(Multi Layer Perceptron). 
# in this net, batch-normalization (bn) is used. 
# bn is very important to stabilize the training of net. 
#torch.manual_seed(0)
class Net(nn.Module): 

    def __init__(self):
        super(Net, self).__init__()

        self.l1 = nn.Linear(dim, 1200)
        self.bn1 = nn.BatchNorm1d(1200)

        self.l2 = nn.Linear(1200,1200)
        self.bn2 = nn.BatchNorm1d(1200)

        self.l3 = nn.Linear(1200,C)
        
    def forward(self, x):
       
        x = F.relu(self.bn1(self.l1(x)))
        x = F.relu(self.bn2(self.l2(x)))
        x = self.l3(x)
        
        return x




net = Net()

# throw net object to gpu
net = net.to(dev)


################################################## Training of imsat ############################################

# decide hyperparameter values for imsat training
epochs = 50 # number of epochs

xi = 10**(1)       # xi is used for making adversarial vector. if xi becomes smaller, 
                   # theoretically obtained r_vadv becomes more priecise


mini_size = 250 # size of mini-batch training dataset

m = X.shape[0]//mini_size # number of iteration at each epoch 


mu = 0.1
gamma = 0.55
eta1 = 5




## define optimizer for set of parameters in deep neural network
## lr is the learning rate of parameter vector, betas are the lr of gradient and second moment of gradient
optimizer = optim.Adam(net.parameters(), 
                        lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)


## tau_pos = 2 or 5 is good for reuters, c=10,20 tau_pos =.75?
tau_pos = 0.01
tau_neg = tau_pos 


# given the number of negative, how much rate of them should be used for trainingm, rate=1
# means 100% used
rate = 1


#itr_num = 0
print("Start training of SIM_agm.")
for epoch in range(epochs):
    print("At ", epoch, "-th epoch, ")

    # set empiricial loss to 0 in the beginning of epoch
    empirical_loss = 0.0

    idx_eph = np.random.permutation(X.shape[0])
    
    net.train()

    for itr in range(m):
      
      ## chose a core idx of mini_batch
      idx_itr = idx_eph[itr*mini_size:(itr+1)*mini_size]

      # define components at each iteration
      X_itr = X[idx_itr,:]


      K0 = 5 #should be smaller than K=200, here we define T(x) for each x, K0 = 5, 10, 15, 20, 25, 50, 100, 150, 200(=K)
      # X_agm1_itr, idx_agm1_itr = get_agm(X, idx_itr, indices[:,1:K0+1])
      X_agm1_itr, idx_agm1_itr = get_agm(X, idx_itr, indices[:,int(2*K0/3):K0+1])
      # X_agm1_itr, idx_agm1_itr = get_agm(X, idx_itr, indices[:,int(K0/2):K0+1])
      


      R_vat = return_vat_Loss(net, X_itr, xi, R[idx_itr,:])
      R_vat1 = return_vat_Loss(net, X_agm1_itr, xi, R[idx_agm1_itr,:])



      soft_out_itr = torch.softmax(net(X_itr) , 1)
  

      ## define positive and negative loss
      l_p, l_n, soft_out_agm1_itr = SiameseLoss(net, soft_out_itr, X_agm1_itr, tau_pos, tau_neg, rate)

      ## define entropy of y
      ent_y = entropy(torch.cat((soft_out_itr, soft_out_agm1_itr),0))

      ## define shannon conditional entropy loss H(p(y|x)) named by c_ent.
      c_ent = conditional_entropy(torch.cat((soft_out_itr, soft_out_agm1_itr),0))
      
              
      # objective of sim
      eta2 = 1
      objective = ((R_vat + R_vat1)/2) - mu*( (1-gamma)*(eta1*ent_y - c_ent) + gamma*eta2*(-l_p - l_n)  )



      # update the set of parameters in deep neural network by minimizing loss
      optimizer.zero_grad() 
      objective.backward()
      optimizer.step()

      empirical_loss = empirical_loss + objective.data



    #empirical_loss = running_loss/m
    empirical_loss = empirical_loss.cpu().numpy()
    print("average empirical loss is", empirical_loss/m, ',')

    net.eval()

    # at each epoch, prediction accuracy is displayed
    with torch.no_grad():
        out = net(X)
        preds = torch.argmax(out, dim=1)
        preds = preds.cpu().numpy()
        preds = preds.reshape(1, preds.shape[0])
        clustering_acc = ReturnACC(preds[0], Y[0], C)
    print("and current clustering accuracy is", clustering_acc )

Start training of SIM_agm.
At  0 -th epoch, 
average empirical loss is -0.07775627772013347 ,
and current clustering accuracy is 0.38315822986310094
At  1 -th epoch, 
average empirical loss is -0.19015328725179037 ,
and current clustering accuracy is 0.383741908097209
At  2 -th epoch, 
average empirical loss is -0.37335840861002606 ,
and current clustering accuracy is 0.3842725246736708
At  3 -th epoch, 
average empirical loss is -0.49891138712565103 ,
and current clustering accuracy is 0.38310516820545476
At  4 -th epoch, 
average empirical loss is -0.548175557454427 ,
and current clustering accuracy is 0.38299904489016234
At  5 -th epoch, 
average empirical loss is -0.5786329142252604 ,
and current clustering accuracy is 0.3821500583678234
At  6 -th epoch, 
average empirical loss is -0.5992598470052083 ,
and current clustering accuracy is 0.3820969967101772
At  7 -th epoch, 
average empirical loss is -0.5982380167643229 ,
and current clustering accuracy is 0.38931338215005834
At  8 -